## COAST Transect Calculations
Exercise for calcualting temperature, velocities and transport along a transect.

#### Scripting - set paths and import modules
Begin with module imports.

In [5]:
import warnings
warnings.filterwarnings('ignore')

import coast
import matplotlib.pyplot as plt

Set model paths

In [2]:
path = "/gws/nopw/j04/jmmp/tmp_slwa/AMM7_SSB_sample/"
path = "/gws/pw/j07/workshop/users/train006/AMM7_SSB_sample/"
model_path_t = path +  "amm7_1d_20040101_20040131_grid_T.nc"
model_path_u = path +  "amm7_1d_20040101_20040131_grid_U.nc"
model_path_v = path +  "amm7_1d_20040101_20040131_grid_V.nc"
jmmp_path = "/gws/nopw/j04/jmmp/"
domcfg_path = path + "amm7_SSB_mesh_mask.nc"

Set coast config paths

In [3]:
cfg_path = "/home/users/ryapat30/NOC/COAsT/config/"
nemo_t_json = cfg_path + "example_nemo_grid_t.json"
nemo_u_json = cfg_path + "example_nemo_grid_u.json"
nemo_v_json = cfg_path + "example_nemo_grid_v.json"
nemo_f_json = cfg_path + "example_nemo_grid_v.json"

Assign NEMO output to Gridded object to each position on the Awarkara C-Grid. These consist of:

- T-Grid for tracer points at the cell centres
- U-Grid for velocity points on the western cell edge
- V-Grid for velocity points on the southern cell edge
- F-Grid for vorticity/Coriolis points on the south-western cell corners

In [7]:
nemo_t = coast.Gridded(model_path_t, domcfg_path, config=nemo_t_json)
nemo_u = coast.Gridded(model_path_u, domcfg_path, config=nemo_u_json)
nemo_v = coast.Gridded(model_path_v, domcfg_path, config=nemo_v_json)
nemo_f = coast.Gridded(fn_domain=domcfg_path, config=nemo_f_json)

AttributeError: 'Gridded' object has no attribute 'config'

Get transects at T and F points.

In [ ]:
tran_t = coast.TransectT(nemo_t, (54, -15), (56, -12))
tran_f = coast.TransectF(nemo_f, (54, -15), (56, -12))

Add velociites normal to to transect.

In [ ]:
tran_f.calc_flow_across_transect(nemo_u, nemo_v)

### Plotting

#### Plot Time-Mean Temperature Along Extracted Transect

Alias temperature variable.

In [ ]:
temp = tran_t.data.temperature

Take time-mean of temperature transect.

In [ ]:
temp_mean = temp.mean(dim="t_dim")

Plot time-mean temperature along transect.

In [ ]:
temp_mean.plot.pcolormesh(y="depth_0", yincrease=False)

#### Plot Time-Mean velocities normal to the transect

Alias normal velocites.

In [ ]:
cross_vel = tran_f.data_cross_tran_flow.normal_velocities

Take time-mean of velocity transect.

In [ ]:
cross_vel_mean = cross_vel.mean(dim="t_dim")

Plot time-mean cross transect velocities

In [ ]:
cross_vel_mean.plot.pcolormesh(yincrease=False, y="depth_0", cbar_kwargs={"label": "m/s"})

#### Plot Time-Mean Volume Transport Accoss Transect

Alias transport normal to transect.

In [ ]:
cross_transport = tran_f.data_cross_tran_flow.normal_transports

Take time mean.

In [ ]:
cross_transport_mean = cross_transport.mean(dim="t_dim")

Plot cross transect transport.

In [ ]:
cross_transport_mean.plot()
plt.ylabel("Sv")

The above creates a noisy field due to a zig-zag path taken by the transect.

In [ ]:
cross_transport_mean